# Extended Kalman Filter

Here we try to apply the [pyro tutorial](https://pyro.ai/examples/ekf.html) on EKF on our data.



In [2]:
import os
import math

import torch
import pyro
import pyro.distributions as dist
from pyro.infer.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO, config_enumerate
from pyro.contrib.tracking.extended_kalman_filter import EKFState
from pyro.contrib.tracking.distributions import EKFDistribution
from pyro.contrib.tracking.dynamic_models import NcvContinuous
from pyro.contrib.tracking.measurements import PositionMeasurement

## Test that the tutorial works

In [7]:
dt = 1e-2
num_frames = 10
dim = 4

# Continuous model
ncv = NcvContinuous(dim, 2.0)

# Truth trajectory
xs_truth = torch.zeros(num_frames, dim)
# initial direction
theta0_truth = 0.0
# initial state
with torch.no_grad():
    xs_truth[0, :] = torch.tensor([0.0, 0.0,  math.cos(theta0_truth), math.sin(theta0_truth)])
    for frame_num in range(1, num_frames):
        # sample independent process noise
        dx = pyro.sample('process_noise_{}'.format(frame_num), ncv.process_noise_dist(dt))
        xs_truth[frame_num, :] = ncv(xs_truth[frame_num-1, :], dt=dt) + dx

In [8]:
# Measurements
measurements = []
mean = torch.zeros(2)
# no correlations
cov = 1e-5 * torch.eye(2)
with torch.no_grad():
    # sample independent measurement noise
    dzs = pyro.sample('dzs', dist.MultivariateNormal(mean, cov).expand((num_frames,)))
    # compute measurement means
    zs = xs_truth[:, :2] + dzs

xs_truth.shape, zs.shape

(torch.Size([10, 4]), torch.Size([10, 2]))

In [9]:
def model(data):
    # a HalfNormal can be used here as well
    R = pyro.sample('pv_cov', dist.HalfCauchy(2e-6)) * torch.eye(4)
    Q = pyro.sample('measurement_cov', dist.HalfCauchy(1e-6)) * torch.eye(2)
    # observe the measurements
    print(data.shape, xs_truth[0].shape, R.shape, Q.shape)
    pyro.sample('track_{}'.format(i), EKFDistribution(xs_truth[0], R, ncv,
                                                      Q, time_steps=num_frames),
                obs=data)

guide = AutoDelta(model)  # MAP estimation

In [15]:
torch.concat((zs, zs))

tensor([[0.4963, 0.7682],
        [0.0885, 0.1320],
        [0.3074, 0.6341],
        [0.4901, 0.8964],
        [0.4556, 0.6323],
        [0.3489, 0.4017],
        [0.0223, 0.1689],
        [0.2939, 0.5185],
        [0.6977, 0.8000],
        [0.1610, 0.2823],
        [0.4963, 0.7682],
        [0.0885, 0.1320],
        [0.3074, 0.6341],
        [0.4901, 0.8964],
        [0.4556, 0.6323],
        [0.3489, 0.4017],
        [0.0223, 0.1689],
        [0.2939, 0.5185],
        [0.6977, 0.8000],
        [0.1610, 0.2823]])

In [16]:
optim = pyro.optim.Adam({'lr': 2e-2})
svi = SVI(model, guide, optim, loss=Trace_ELBO(retain_graph=True))

pyro.set_rng_seed(0)
pyro.clear_param_store()

zs = torch.rand_like(zs)

smoke_test = False
for i in range(250 if not smoke_test else 2):
    loss = svi.step(torch.concat((zs, zs)))
    if not i % 10:
        print('loss: ', loss)

torch.Size([20, 2]) torch.Size([4]) torch.Size([4, 4]) torch.Size([2, 2])


AssertionError: value shape: torch.Size([20, 2]), event_shape: (10, 2)

In [6]:
# retrieve states for visualization
R = guide()['pv_cov'] * torch.eye(4)
Q = guide()['measurement_cov'] * torch.eye(2)
ekf_dist = EKFDistribution(xs_truth[0], R, ncv, Q, time_steps=num_frames)
states= ekf_dist.filter_states(zs)

In [8]:
[s.mean for s in states]

[tensor([-0.0029,  0.0037,  1.0000,  0.0000], grad_fn=<SubBackward0>),
 tensor([ 0.0066, -0.0015, -0.4857, -0.0077], grad_fn=<SubBackward0>),
 tensor([0.0229, 0.0005, 0.1597, 0.0048], grad_fn=<SubBackward0>),
 tensor([ 0.0308,  0.0021, -0.0330,  0.0008], grad_fn=<SubBackward0>),
 tensor([ 4.3259e-02, -3.6855e-05,  2.4640e-02, -2.9565e-03],
        grad_fn=<SubBackward0>),
 tensor([0.0497, 0.0012, 0.0016, 0.0024], grad_fn=<SubBackward0>),
 tensor([0.0563, 0.0073, 0.0079, 0.0071], grad_fn=<SubBackward0>),
 tensor([ 0.0686,  0.0074,  0.0134, -0.0018], grad_fn=<SubBackward0>),
 tensor([ 0.0770,  0.0024,  0.0071, -0.0058], grad_fn=<SubBackward0>),
 tensor([ 0.0840, -0.0044,  0.0070, -0.0072], grad_fn=<SubBackward0>)]

## Try with own data

### Data

In [3]:
from main import get_energy_data, get_weather_data
import pandas as pd

df = pd.read_csv("preprocessed_data/df.csv").iloc[:1000]
df["time_str"] = [d.split("+")[0] for d in df["time_str"]]
df["time_str"] = pd.to_datetime(df["time_str"], infer_datetime_format=True)#'%Y-%m-%d %H:%M:%S.%f') # 2015-01-01 10:00:00+00:00
dfW = get_weather_data(df)
dfE = get_energy_data(df)

X_W = torch.from_numpy(dfW.values).float()
X_E = torch.from_numpy(dfE.values).float()

obs = torch.from_numpy(df["price actual"].values).float()

### Model

In [40]:
h_dim = 10
h_dim = X_E.shape[1]
dyn_model = NcvContinuous(h_dim, 2.0)
T = 10

def model(obs = None):
    # a HalfNormal can be used here as well
    R = pyro.sample('pv_cov', dist.HalfCauchy(2e-6)) * torch.eye(h_dim)
    Q = pyro.sample('measurement_cov', dist.HalfCauchy(1e-6)) * torch.eye(1)

    # h0 = pyro.sample(f'h0', dist.Normal(torch.zeros(h_dim), torch.ones(h_dim)))
    print(obs.shape, X_E[0].shape, R.shape, Q.shape)
    # observe the measurements
    y = pyro.sample('track_{}'.format(i), EKFDistribution(X_E[0], R, dyn_model,
                                                      Q, time_steps=T),
                obs=obs)

    return y

guide = AutoDelta(model)  # MAP estimation

### Inference

In [41]:
optim = pyro.optim.Adam({'lr': 2e-2})
svi = SVI(model, guide, optim, loss=Trace_ELBO(retain_graph=True))

pyro.set_rng_seed(0)
pyro.clear_param_store()

x = torch.hstack((obs[:T], obs[:T], obs[:T], obs[:T], obs[:T])).reshape(T, 5)

for i in range(10):
    loss = svi.step(obs[:T])
    if not i % 10:
        print('loss: ', loss)

        #torch.Size([10, 2]) torch.Size([4]) torch.Size([4, 4]) torch.Size([2, 2])

torch.Size([10]) torch.Size([11]) torch.Size([11, 11]) torch.Size([1, 1])


AssertionError: position and velocity vectors must be the same dimension